# 学习分类文本

模式识别是自然语言处理的一个核心部分。以-ed 结尾的词往往是过去时态动词（第 5
章）。频繁使用 will 是新闻文本的暗示（第 3 章）。这些可观察到的模式——词的结构和词
频——恰好与特定方面的含义关联，如：时态和主题。但我们怎么知道从哪里开始寻找，形
式的哪一方面关联含义的哪一方面？
本章的目标是要回答下列问题：
1. 我们怎样才能识别语言数据中能明显用于对其分类的特征？
2. 我们怎样才能构建语言模型，用于自动执行语言处理任务？
3. 从这些模型中我们可以学到哪些关于语言的知识？
一路上，我们将研究一些重要的机器学习技术，包括决策树、朴素贝叶斯分类器和最大
熵分类。我们会掩盖这些技术的数学和统计的基础，集中关注如何以及何时使用它们（更多
的技术背景知识见 6.9 节）。在看这些方法之前，我们首先需要知道这个主题的范围十分广


## 选取特征

特征提取器返回字典，这个字典被称为特征集。然后利用

nltk自带的朴素贝叶斯分类器 NaiveBayesClassifier 生成分类器。并且可以用nltk.classify.accuracy(分类器,测试集) 测试准确度

In [53]:
import nltk
from nltk.corpus import names
import random

def gender_features(word): #特征提取器
    return {'last_letter':word[-1]} #特征集就是最后一个字母

names = [(name,'male') for name in names.words('male.txt')]+[(name,'female') for name in names.words('female.txt')]
random.shuffle(names)#将序列打乱

features = [(gender_features(n),g) for (n,g) in names]#返回对应的特征和标签

train,test = features[500:],features[:500] #训练集和测试集
classifier = nltk.NaiveBayesClassifier.train(train) #生成分类器

print('Neo is a',classifier.classify(gender_features('Neo')))#分类

print(nltk.classify.accuracy(classifier,test)) #测试准确度

classifier.show_most_informative_features(5)#得到似然比，检测对于哪些特征有用

('Neo is a', 'male')
0.752
Most Informative Features
             last_letter = u'a'           female : male   =     35.8 : 1.0
             last_letter = u'k'             male : female =     30.7 : 1.0
             last_letter = u'f'             male : female =     16.6 : 1.0
             last_letter = u'p'             male : female =     12.5 : 1.0
             last_letter = u'd'             male : female =     10.0 : 1.0


In [54]:
# 当我们训练大的语料库的时候，链表会占用很大内存。这时候nltk提供了：apply_features，会生成链表，但是不会在内存中存储所有对象。
# no ： features = [(gender_features(n),g) for (n,g) in names]
from nltk.classify import apply_features
train_set = apply_features(gender_features,names[500:])
test_set = apply_features(gender_features,names[:500])

过拟合：当特征过多

当特征过多的时候（特征集的键值过多），会对一般化的新例子不起作用，称为过拟合。如果抉择特征集的大小，需要不停的测试，找到最吻合的特征集。
错误分析

为了使特征提取器准确度更高，一般将源数据分为两大部分，三小部分：

    开发集：
        训练集：负责开发
        开发测试集：负责错误分析
    测试集：负责最终评估

In [55]:
# 特征提取器，过拟合提取特征。这里请对比gender_features2与上一版本的区别
def gender_features2(name):
    features = {}
    features["firstletter"] = name[0].lower()
    features["lastletter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count(%s)" % letter] = name.lower().count(letter)
        features["has(%s)" % letter] = (letter in name.lower())
    return features

featuresets = [(gender_features2(n), g) for (n,g) in names]
# list(featuresets)
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
# 对比上一版本的预测结果，基本可以发现精确度下降了b
nltk.classify.accuracy(classifier, test_set)

0.752

In [56]:
classifier.show_most_informative_features(5)

Most Informative Features
              lastletter = u'a'           female : male   =     35.8 : 1.0
              lastletter = u'k'             male : female =     30.7 : 1.0
              lastletter = u'f'             male : female =     16.6 : 1.0
              lastletter = u'p'             male : female =     12.5 : 1.0
              lastletter = u'd'             male : female =     10.0 : 1.0


In [57]:
gender_features2('John')

{'count(a)': 0,
 'count(b)': 0,
 'count(c)': 0,
 'count(d)': 0,
 'count(e)': 0,
 'count(f)': 0,
 'count(g)': 0,
 'count(h)': 1,
 'count(i)': 0,
 'count(j)': 1,
 'count(k)': 0,
 'count(l)': 0,
 'count(m)': 0,
 'count(n)': 1,
 'count(o)': 1,
 'count(p)': 0,
 'count(q)': 0,
 'count(r)': 0,
 'count(s)': 0,
 'count(t)': 0,
 'count(u)': 0,
 'count(v)': 0,
 'count(w)': 0,
 'count(x)': 0,
 'count(y)': 0,
 'count(z)': 0,
 'firstletter': 'j',
 'has(a)': False,
 'has(b)': False,
 'has(c)': False,
 'has(d)': False,
 'has(e)': False,
 'has(f)': False,
 'has(g)': False,
 'has(h)': True,
 'has(i)': False,
 'has(j)': True,
 'has(k)': False,
 'has(l)': False,
 'has(m)': False,
 'has(n)': True,
 'has(o)': True,
 'has(p)': False,
 'has(q)': False,
 'has(r)': False,
 'has(s)': False,
 'has(t)': False,
 'has(u)': False,
 'has(v)': False,
 'has(w)': False,
 'has(x)': False,
 'has(y)': False,
 'has(z)': False,
 'lastletter': 'n'}

错误分析

为了使特征提取器准确度更高，一般将源数据分为两大部分，三小部分：

    开发集：
        训练集：负责开发
        开发测试集：负责错误分析
    测试集：负责最终评估

In [58]:
train_names = names[1500:]
devtest_names = names[500:1500]
test_names = names[:500]

train_set = [(gender_features(n),g) for (n,g) in train_names]
devtest_set = [(gender_features(n),g) for (n,g) in devtest_names]
test_set = [(gender_features(n),g) for (n,g) in test_names]

In [59]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier,devtest_set))

0.747


In [60]:
######################记录报错的案例###############################
errors = []
for (name,tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess!=tag:
        errors.append((tag,guess,name))
##################################################################

In [61]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    print 'correct=%-8s guess=%-8s name=%-30s'%(tag, guess, name)

correct=female   guess=male     name=Addis                         
correct=female   guess=male     name=Adriaens                      
correct=female   guess=male     name=Agnes                         
correct=female   guess=male     name=Allis                         
correct=female   guess=male     name=Allyn                         
correct=female   guess=male     name=Allys                         
correct=female   guess=male     name=Anett                         
correct=female   guess=male     name=Beret                         
correct=female   guess=male     name=Bette-Ann                     
correct=female   guess=male     name=Bidget                        
correct=female   guess=male     name=Bliss                         
correct=female   guess=male     name=Brigit                        
correct=female   guess=male     name=Caro                          
correct=female   guess=male     name=Carolynn                      
correct=female   guess=male     name=Cathleen   

In [62]:
'''
浏览这个错误列表，它明确指出一些多个字母的后缀可以指示名字性别。
例如，yn结尾的名字显示以女性为主，尽管事实上，n结尾的名字往往是男性；以ch结尾的名字通常是男性，
尽管以h结尾的名字倾向于是女性。因此，调整我们的特征提取器包括两个字母后缀的特征：
'''
def gender_features(word):
    return {'suffix1': word[-1:],
            'suffix2': word[-2:]}

In [63]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.797


## 文档分类

这里的分类标签选成词汇，通过对文本前N个词的观察，得到预测标签。

In [64]:
>>> from nltk.corpus import movie_reviews
>>> documents = [(list(movie_reviews.words(fileid)), category)
...              for category in movie_reviews.categories()
...              for fileid in movie_reviews.fileids(category)]
>>> random.shuffle(documents)

In [78]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]#freq word features
'''
接下来，我们为文档定义一个特征提取器，这样分类器就会知道哪些方面的数据应注意\）。
对于文档主题识别，我们可以为每个词定义一个特性表示该文档是否包含这个词。
为了限制分类器需要处理的特征的数目，
我们一开始构建一个整个语料库中前2000个最频繁词的列表[1]。
然后，定义一个特征提取器，简单地检查这些词是否在一个给定的文档中。

'''
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:#document word in features
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [81]:
print document_features(movie_reviews.words('pos/cv957_8737.txt'))

{'contains(corporate)': False, 'contains(barred)': False, 'contains(batmans)': False, 'contains(menacing)': False, 'contains(rags)': False, 'contains(compton)': False, 'contains(inquires)': False, 'contains(nosebleeding)': False, 'contains(playhouse)': False, 'contains(peculiarities)': False, 'contains(guarded)': False, 'contains(kilgore)': False, 'contains(tarnish)': False, 'contains(sand)': False, 'contains(busting)': False, 'contains(wedge)': False, 'contains(smelling)': False, 'contains(tulip)': False, 'contains(singled)': False, 'contains(wahlberg)': False, 'contains(needed)': False, 'contains(lydia)': False, 'contains(rick)': False, 'contains(cambodia)': False, 'contains(grandma)': False, 'contains(jovivich)': False, 'contains(pinon)': False, 'contains(fix)': False, 'contains(marla)': False, 'contains(resources)': False, 'contains(nomi)': False, 'contains(irs)': False, 'contains(mason)': False, 'contains(vicariously)': False, 'contains(ingrained)': False, 'contains(skepticism)': 

In [83]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [84]:
print nltk.classify.accuracy(classifier, test_set)

0.59


In [85]:
classifier.show_most_informative_features(5)

Most Informative Features
          contains(sans) = True              neg : pos    =      9.0 : 1.0
    contains(mediocrity) = True              neg : pos    =      7.7 : 1.0
   contains(bruckheimer) = True              neg : pos    =      6.3 : 1.0
   contains(overwhelmed) = True              pos : neg    =      6.3 : 1.0
     contains(uplifting) = True              pos : neg    =      5.8 : 1.0


In [80]:
print documents[0]#可以看到最后是neg

([u'if', u'you', u"'", u're', u'into', u'watching', u'near', u'on', u'two', u'hours', u'of', u'bored', u',', u'foul', u'-', u'mouthed', u'florida', u'teens', u'having', u'sex', u',', u'doing', u'drugs', u',', u'having', u'sex', u',', u'listening', u'to', u'eminem', u',', u'having', u'sex', u',', u'playing', u'video', u'games', u',', u'having', u'sex', u',', u'and', u'killing', u'one', u'of', u'their', u'peers', u',', u'then', u'"', u'bully', u'"', u"'", u's', u'for', u'you', u'.', u'based', u'on', u'jim', u'schutze', u"'", u's', u'novelization', u'of', u'a', u'true', u'-', u'life', u'event', u',', u'"', u'bully', u'"', u'charts', u'the', u'story', u'of', u'a', u'handful', u'of', u'disenchanted', u'teenagers', u'who', u',', u'in', u'1993', u',', u'murdered', u'their', u'high', u'school', u'bully', u'in', u'cold', u',', u'calculated', u'blood', u'.', u'the', u'film', u'could', u'have', u'provided', u'fascinating', u'insights', u'into', u'what', u'turned', u'these', u'aimless', u'kids', u

## 词性标注

In [89]:
'''
第5.中，我们建立了一个正则表达式标注器，通过查找词内部的组成，为词选择词性标记。
然而，这个正则表达式标注器是手工制作的。作为替代，我们可以训练一个分类器来算出哪个后缀最有信息量。
首先，让我们找出最常见的后缀：
'''
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

In [90]:
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]

In [91]:
print common_suffixes

[u'e', u',', u'.', u's', u'd', u't', u'he', u'n', u'a', u'of', u'the', u'y', u'r', u'to', u'in', u'f', u'o', u'ed', u'nd', u'is', u'on', u'l', u'g', u'and', u'ng', u'er', u'as', u'ing', u'h', u'at', u'es', u'or', u're', u'it', u'``', u'an', u"''", u'm', u';', u'i', u'ly', u'ion', u'en', u'al', u'?', u'nt', u'be', u'hat', u'st', u'his', u'th', u'll', u'le', u'ce', u'by', u'ts', u'me', u've', u"'", u'se', u'ut', u'was', u'for', u'ent', u'ch', u'k', u'w', u'ld', u'`', u'rs', u'ted', u'ere', u'her', u'ne', u'ns', u'ith', u'ad', u'ry', u')', u'(', u'te', u'--', u'ay', u'ty', u'ot', u'p', u'nce', u"'s", u'ter', u'om', u'ss', u':', u'we', u'are', u'c', u'ers', u'uld', u'had', u'so', u'ey']


In [92]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

In [93]:
'''
特征提取函数的行为就像有色眼镜一样，强调我们的数据中的某些属性（颜色），并使其无法看到其他属性。
分类器在决定如何标记输入时，将完全依赖它们强调的属性。
在这种情况下，分类器将只基于一个给定的词拥有（如果有）哪个常见后缀的信息来做决定。
现在，我们已经定义了我们的特征提取器，可以用它来训练一个新的“决策树”的分类器
'''
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]

In [94]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]

In [96]:
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.6270512182993535

In [97]:
classifier.classify(pos_features('cats'))

u'NNS'

## 探索上下文语境
为了采取基于词的上下文的特征，我们必须修改以前为我们的特征提取器定义的模式。

不是只传递已标注的词，我们将传递整个（未标注的）句子，以及目标词的索引

In [98]:
def pos_features(sentence, i):
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features

In [99]:
pos_features(brown.sents()[0], 8)

{'prev-word': u'an',
 'suffix(1)': u'n',
 'suffix(2)': u'on',
 'suffix(3)': u'ion'}

In [100]:
'''
enumerate:
>>>seasons = ['Spring', 'Summer', 'Fall', 'Winter']
>>> list(enumerate(seasons))
[(0, 'Spring'), (1, 'Summer'), (2, 'Fall'), (3, 'Winter')]
'''
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
	untagged_sent = nltk.tag.untag(tagged_sent)
	for i, (word, tag) in enumerate(tagged_sent):
		featuresets.append((pos_features(untagged_sent, i), tag))
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.7891596220785678

## 序列分类

In [101]:
#带有历史的特征提取器
def pos_features(sentence, i, history):
    features = {"suffix(1)": sentence[i][-1:],
                 "suffix(2)": sentence[i][-2:],
                 "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
        features["prev-tag"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
        features["prev-tag"] = history[i-1]
    return features
# history中的每个标记对应sentence中的一个词。但是请注意，history将只包含我们已经归类的词的标记，也就是目标词左侧的词。

In [102]:
class ConsecutivePosTagger(nltk.TaggerI): #这里定义新的选择器类，继承nltk.TaggerI
    def __init__(self,train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_set = nltk.tag.untag(tagged_sent) #去标签化
            history = []
            for i,(word,tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_set,i,history)
                history.append(tag) #将tag添加进去
                train_set.append((featureset,tag)) #拿到了训练集
            self.classifier = nltk.NaiveBayesClassifier.train(train_set) #创建训练模型

    def tag(self,sentence): #必须定义tag方法
        history = []
        for i,word in enumerate(sentence):
            featureset = pos_features(sentence,i,history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence,history)

In [103]:
tagged_sents = brown.tagged_sents(categories="news")
size = int(len(tagged_sents)*0.1)
train_sents,test_sents = tagged_sents[size:],tagged_sents[:size]

In [105]:
print(train_sents[0])

[(u'He', u'PPS'), (u'assured', u'VBD'), (u'Mr.', u'NP'), (u'Martinelli', u'NP'), (u'and', u'CC'), (u'the', u'AT'), (u'council', u'NN'), (u'that', u'CS'), (u'he', u'PPS'), (u'would', u'MD'), (u'study', u'VB'), (u'the', u'AT'), (u'correct', u'JJ'), (u'method', u'NN'), (u'and', u'CC'), (u'report', u'VB'), (u'back', u'RB'), (u'to', u'IN'), (u'the', u'AT'), (u'council', u'NN'), (u'as', u'QL'), (u'soon', u'RB'), (u'as', u'CS'), (u'possible', u'JJ'), (u'.', u'.')]


In [106]:
tagger = ConsecutivePosTagger(train_sents)

In [107]:
print(tagger.evaluate(test_sents))

0.798052851182


## 其他序列分类方法

### 1.brill标注器：

如果我们决定将一个词标

注为名词，但后来发现的证据表明应该是一个动词，就没有办法回去修复我们的错误。这个

问题的一个解决方案是采取转型策略。转型联合分类的工作原理是为输入的标签创建一个初

始值，然后反复提炼那个值，尝试修复相关输入之间的不一致

### 2.隐马尔可夫模型：

类似于连续分类器，它不光看输入也看已预测标记的历

史。然而，不是简单地找出一个给定的词的单个最好的标签，而是为标记产生一个概率分布。

然后将这些概率结合起来计算标记序列的概率得分，最高概率的标记序列会被选中

这种模型的缺点就是计算量相当大，可以采用动态规划来解决这种问题。

基于这种模型，产生了最大熵马尔可夫模型和线性链条件随机场模型，二者为可能序列打分的算法不同。

##  句子分割

句子分割可以看作是一个标点符号的分类任务：每当我们遇到一个可能会结束一个句子

的符号，如句号或问号，我们必须决定它是否终止了当前句子。

In [108]:
sents = nltk.corpus.treebank_raw.sents()
# 单独的句子标识符的合并链表
tokens = []
# 所有句子边界标识索引 如 . ? ! , :等
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset-1)
    
# 特征提取器:我们需要指定用于决定标点是否表示句子边界的数据特征
def punct_features(tokens, i):
	return {'next-word-capitalized': tokens[i+1][0].isupper(),
            'prevword': tokens[i-1].lower(),
            'punct': tokens[i],
            'prev-word-is-one-char': len(tokens[i-1]) == 1}

featuresets = [(punct_features(tokens, i), (i in boundaries))
				for i in range(1, len(tokens)-1)
				if tokens[i] in '.?!']
# 进行训练，检验模型准确率
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.936026936026936

## 识别对话行为类型

In [109]:
# 提取即时消息会话语料库
posts = nltk.corpus.nps_chat.xml_posts()[:10000]
# 特征提取器，每个帖子包含什么词
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
    	features['contains(%s)' % word.lower()] = True
    return features
# 构建模型，判断是否为对话行为
featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.668

## 识别文字蕴含
略


## 测试集

因为调用 random.shuffle()，测试集中包含来自训练使用过的相同的文档的句子。如果文档中有相容的模式（也

就是说，如果一个给定的词与特定词性标记一起出现特别频繁），那么这种差异将体现在开

发集和测试集。一个稍好的做法是确保训练集和测试集来自不同的文件

## 准确度
nltk.classify.accuracy()函数会在给定的测试集上计算分类器

模型的准确度
>classifier = nltk.NaiveBayesClassifier.train(train_set)

>print 'Accuracy: %4.2f' % nltk.classify.accuracy(classifier, test_set)

>0.75

## 精确度和召回率

准确度：测量测试集上分类器正确标注的输入的比例。

精确度（Precision），表示我们发现的项目中有多少是相关的，TP/(TP+ FP)。

召回率（Recall），表示相关的项目中我们发现了多少，TP/(TP+ FN)。

F- 度量值（F-Measure）（或 F-得分，F-Score），组合精确度和召回率为一个单独的得分，被定义为精确度和召回率的调和平均数(2 × Precision × Recall)/(Precision+Recall)。

## 混淆矩阵

混淆矩阵：其中每个 cells[i,j]表示正确的标签 i 被预测为标签 j 的次数 。

因此，对角线项目（即 cells[i,i]）表示正确预测的标签，非对角线项目表示错误。

nltk中可以使用nltk.ConfusionMatrix函数

## 交叉验证

为了评估我们的模型，我们必须为测试集保留一部分已标注的数据。

采用交叉验证的优势是，它可以让我们研究不同的训练集上性能变

化有多大。如果我们从所有 N 个训练集得到非常相似的分数，然后我们可以相当有信心，

得分是准确的。另一方面，如果 N 个训练集上分数很大不同，那么，我们应该对评估得分

的准确性持怀疑态度

## 零计数和平滑
### 零计数：

最简单的方法计算 P(f|label)，特征 f 对标签 label 的标签可能性的贡献，是取得具有给

定特征和给定标签的训练实例的百分比：

 P(f|label) = count(f, label)/count(label)

然而，当训练集中有特征从来没有和给定标签一起出现时，这种简单的方法会产生一个

问题。在这种情况下，我们的 P(f|label)计算值将是 0，这将导致给定标签的标签可能性为 0

### 平滑：

因此，建立朴素贝叶斯模型时，我们通常采用更复杂

的技术，被称为平滑技术，用于计算 P(f|label)，给定标签的特征的概率。例如：给定标签的

一个特征的概率的期望似然估计基本上给每个 count(f,label)值加 0.5，Heldout Heldout Heldout Heldout 估计使用一个

heldout 语料库计算特征频率与特征概率之间的关系。nltk.probability 模块提供了多种平

滑技术的支持

## 最大熵模型

最大熵分类器：与朴素贝叶斯类似，使用搜索技术找出一组将最大限度地提高分类器性能的参数。避免使用广义
迭代缩放（Generalized Iterative Scaling，GIS）或改进的迭代缩放（Improved Iterative Scaling，IIS），这两者都比共轭梯度（Conjugate Gradient，CG ）和 BFGS 优化方法慢很多

联合特征：每个接收它自己的参数的标签和特征的组合。联合特征是有标签的的值的属性，而（简单）特征是未加标签的值的属性。

一般情况下， 最大熵原理是说在与我们所知道的一致的的分布中，我们会选择熵最高（分布最均匀）的。

对于每个联合特征，最大熵模型计算该特征的“经验频率”——即它出现在训练集中的频率。然后，它搜索熵最大的分布，同时也预测每个联合特征正确的频率。

一般情况下，生成式模型确实比条件式模型强大。但是前者所需的参数会大于后者。

## 结论
    决策树可以自动地构建树结构的流程图，用于为输入变量值基于它们的特征加标签，虽
    然它们易于解释，但不适合处理特性值在决定合适标签过程中相互影响的情况。

    在朴素贝叶斯分类器中，每个特征决定应该使用哪个标签的贡献是独立的。它允许特征
    值间有关联，但当两个或更多的特征高度相关时将会有问题。

    最大熵分类器使用的基本模型与朴素贝叶斯相似；不过，它们使用了迭代优化来寻找使
    训练集的概率最大化的特征权值集合。